In [92]:
import requests
import json
import time
import os
from bs4 import BeautifulSoup
!pip install unicodedata2
from unicodedata2 import normalize
from parse_tools import get_page_data

In [101]:
stream = 'https://meduza.io/api/v3/search?chrono=news&locale=ru&page={page}&per_page=30'
USER_AGENT = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0"

In [102]:
%run pipe_tools.py

In [103]:
def get_page_data(page, stream, user_agent, site_url="https://meduza.io/{0}"):
    #print(page)
    headers = {'User-agent' : user_agent}
    ans = requests.get(stream.format(page=page), headers=headers).json()
    urls = [site_url.format(key) for key in ans['documents'].keys()]
    for url in urls:
        page_responce = requests.get(url, headers=headers, timeout=5)

        if page_responce.status_code == 200:
            page_content = BeautifulSoup(page_responce.content, "html.parser")
            #print(page_content)
            textContent = []
            title = page_content.find_all("h1")[0].text
            link = page_content.find_all("meta", property="og:url")[0]["content"]
            tags = page_content.find_all("meta", attrs={"name":"keywords"})[0]["content"]
            category = page_content.find_all(class_="Layout-root")[0]["data-material"]
            for i in range(0, len(page_content.find_all("p"))):
                paragraphs = page_content.find_all("p")[i].text
                textContent.append(normalize('NFKD', paragraphs))

            body = "".join(textContent)
            
            new_title = normalize('NFKD', title)

            entry = {"article_id" : link, "title" : new_title, "category" : category, "tags" : tags, "text" : body}
            
            #print(entry)

            with open('meduza_dump/page{pagenum:03d}_{timestamp}.json'.format(pagenum = page, timestamp = int(time.time())), 'w', encoding='utf-8') as f: 
                json.dump(entry, f, ensure_ascii=False, indent=2)
            #data = json.load(open('res_dump2/page.json', "rb"))
            #print("ss")
            #print(data)
            #print("ss")
            #data['catalog'].append(entry)
            #print(data)
            #json.dump(data, open('res_dump2/page77.json', "wb"), ensure_ascii=False, indent=2)
           
                

In [104]:
%job [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(25))]
%job [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(25,50))]
%job [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(50,75))]
%job [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(75,100))]

#i = get_page_data(page=26, stream=stream, user_agent=USER_AGENT)
#print(i)
#print(i.find_all(class_="Layout-root")[0]["data-material"])

IntProgress(value=0, max=25)

IntProgress(value=0, max=25)

IntProgress(value=0, max=25)

IntProgress(value=0, max=25)

{'article_id': 'https://meduza.io/news/2021/12/27/v-podmoskovie-vynesen-prigovor-po-delu-ob-ubiystve-v-drake-spetsnazovtsa-gru-obvinyaemye-poluchili-ot-pyati-do-20-let-kolonii', 'title': 'В Подмосковье вынесен приговор по делу об убийстве в драке спецназовца ГРУ. Обвиняемые получили от пяти до 20 лет колонии', 'category': 'news', 'tags': 'новости, политика, расследование, репортаж, интервью, реакция, исследование, опрос, тест, президент, приговор, доллар, нефть, экономика, цены, средства, информация, компания, фото, видео, украина, сепаратисты, война, краткий пересказ, бестселлер', 'text': 'Данное сообщение (материал) создано и (или) распространено иностранным средством массовой информации, выполняющим функции иностранного агента, и (или) российским юридическим лицом, выполняющим функции иностранного агента.Нам нужна ваша помощь. Пожалуйста, поддержите «Медузу».Московский областной суд приговорил девятерых фигурантов дела об убийстве бывшего спецназовца ГРУ Никиты Белянкина лето

In [145]:
%job [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(1000,1177))]

IntProgress(value=0, max=177)

In [151]:
jobs.status()

Running jobs:
10 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(1000,1177))]

Completed jobs:
1 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(25,50))]
2 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(50,75))]
4 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(25))]
6 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(325,450))]
9 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(920,1000))]

Dead jobs:
0 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(25))]
3 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_progress(range(75,100))]
5 : [get_page_data(page=page, stream=stream, user_agent=USER_AGENT) for page in log_prog

In [152]:
with open("merged_meduza_dump.json", "wb") as outfile:
    outfile.write("[".encode())
    counter=1
    for f in glob.glob("./meduza_dump/*.json"):
        with open(f, "rb") as infile:
            line = None
            for line in infile:
                outfile.write(line)
            if line is not None and not line.endswith(b"\n"):
                outfile.write(b"\n")
            if counter < len(glob.glob("*.json")):
                outfile.write(",".encode())
            else:
                outfile.write("]".encode())
            counter=counter+1